In [ ]:
import os,random
import keras
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import *
from keras.optimizers import adam
from keras.optimizers import adagrad
import matplotlib
import keras.backend.tensorflow_backend as tfback
import h5py
import pandas as pd
from keras.utils import to_categorical
from keras import backend as K
import tensorflow as tf

In [ ]:
#data processing
#loading data
f = h5py.File("../modulation-prediction/data.hdf5", 'r')
X_train = np.array(f['train'])
X_train = X_train.reshape([X_train.shape[0], 2, -1])

X_test = np.array(f['test'])
X_test = X_test.reshape(X_test.shape[0], 2, -1)

Y_train = pd.read_csv('../modulation-prediction/train_labels.csv').to_numpy()

print(Y_train.shape)
print(X_train.shape)

In [ ]:
class_index = {
 'FM': 0,
 'OQPSK':1,
 'BPSK':2,
 '8PSK':3,
 'AM-SSB-SC':4,
 '4ASK':5,
 '16PSK':6,
 'AM-DSB-SC':7, 
 'QPSK': 8, 
 'OOK':9,
}
index_class = {
 0:'FM',
 1:'OQPSK',
 2:'BPSK',
 3:'8PSK',
 4:'AM-SSB-SC',
 5:'4ASK',
 6:'16PSK',
 7:'AM-DSB-SC', 
 8:'QPSK', 
 9:'OOK',
}

classnum = 10
def classToIndex(catg):
    return class_index[catg]

#direct mapping, not one-hot
def setToNum(dataset):
    tensor = np.zeros([dataset.shape[0],1])
    for li, catg in enumerate(dataset[:,1]):
        tensor[li] = classToIndex(catg)
    return tensor
# class to name
def classToName(cls):
    return index_class[cls]

In [ ]:
Y_number = setToNum(Y_train)
print(Y_number.shape)
Y_train_onehot = to_categorical(Y_number)
print(Y_train_onehot)

In [ ]:
in_shp = [2,1024]
classes = 10
dr = 0.6 # dropout rate (%)
model = models.Sequential()
model.add(Reshape([1]+in_shp, input_shape=in_shp))
model.add(ZeroPadding2D((0, 2), data_format="channels_first"))
model.add(Conv2D(kernel_initializer="glorot_uniform", name="conv1", activation="relu", data_format="channels_first", padding="valid", filters=256, kernel_size=(1, 3)))
model.add(Dropout(dr))

model.add(ZeroPadding2D((0, 2), data_format="channels_first"))
model.add(Conv2D(kernel_initializer="glorot_uniform", name="conv2", activation="relu", data_format="channels_first", padding="valid", filters=256, kernel_size=(2, 3)))
model.add(Dropout(dr))

model.add(ZeroPadding2D((0, 2), data_format="channels_first"))
model.add(Conv2D(kernel_initializer="glorot_uniform", name="conv3", activation="relu", data_format="channels_first", padding="valid", filters=80, kernel_size=(1, 3)))
model.add(Dropout(dr))

model.add(ZeroPadding2D((0, 2), data_format="channels_first"))
model.add(Conv2D(kernel_initializer="glorot_uniform", name="conv4", activation="relu", data_format="channels_first", padding="valid", filters=80, kernel_size=(1, 3)))
model.add(Dropout(dr))
model.add(ZeroPadding2D((0, 2), data_format="channels_first"))

model.add(Flatten())
model.add(Reshape((1,-1)))
model.add(keras.layers.LSTM(50))
model.add(Dense(256, activation="relu", name="dense1", kernel_initializer="he_normal"))
model.add(Dropout(dr))
model.add(Dense(classes, name="dense2", kernel_initializer="he_normal"))
model.add(Activation('softmax'))
model.add(Reshape([classes]))
#opt=adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

In [ ]:
# Set up some params 
nb_epoch = 1000     # number of epochs to train on
batch_size = 32  # training batch size

filepath = '../modulation-prediction/'
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])
history = model.fit(X_train,
    Y_train_onehot,
    batch_size=batch_size,
    epochs=nb_epoch,
    verbose=2)
# we re-load the best weights once training is finished
model.load_weights(filepath)

In [ ]:
test_Y_hat = model.predict(X_test, batch_size=1024)
# print(test_Y_hat)
predictions = np.argmax(test_Y_hat, axis=1)
print(predictions)

predictionlist = predictions.tolist()
names = []
for line in predictionlist:
    names.append(classToName(line))
    
outputs = np.asarray(names, dtype = str)
categories = pd.DataFrame(outputs, columns = ['Category'])       
categories.to_csv('categories.csv')